# 利用py注解实现创建sql

本文只提供了最基本的定制sql的注解，但是实际中，你可以通过这些基本注解，再定义自己的注解，从而实现更丰富的功能。

In [9]:
from etl import *

# 测试select和subquery

In [18]:
def test_Select_and_SubQuery():
    """

    :return:
    """

    @SubQuery(field=['id', 'name'], alias='t1')
    @Select(field=['*'], f=["1=1", 'and', 'id >100'])
    def table():
        return 'table'

    print('---------------------')
    print('| Select and SubQuery')
    print('---------------------')
    s = table()
    print('s:   {}'.format(s))

In [19]:
test_Select_and_SubQuery()

---------------------
| Select and SubQuery
---------------------
s:   select id,name from (select * from table WHERE 1=1 and id >100) t1


# 测试join

In [20]:
def test_Join():
    """

    :return:
    """

    @Select(field=['*'], f=["1=1"])
    def table1():
        return 'table1'

    @Select(field=['*'], f=["1=1"])
    def table2():
        return 'table2'

    print('---------------------')
    print('| Join')
    print('---------------------')
    s1 = table1()
    s2 = table2()
    print('s1:  {}'.format(s1))
    print('s2:  {}'.format(s2))

    @Join(t="right", before=s1, after=s2, field=['*'], on=['t1.id=t2.id'])
    def join(before_field=[], after_field=[]):
        return {
            "before_field": before_field,
            "after_field": after_field
        }

    s3 = join()
    print('s3:  {}'.format(s3))

In [21]:
test_Join()

---------------------
| Join
---------------------
s1:  select * from table1 WHERE 1=1
s2:  select * from table2 WHERE 1=1
s3:  
    select * from (select * from table1 WHERE 1=1) t1
    right join (select * from table2 WHERE 1=1) t2 on(t1.id=t2.id)
    


# 测试selectmap和map

In [22]:
def test_SelectMap_and_Map():
    """

    :return:
    """
    print('---------------------')
    print('| SelectMap and Map')
    print('---------------------')

    @SelectMap(
        field=['identity', 'start_date', 'len', "regexp_replace(regexp_replace(y, '\\\\]', ''), '\\\\[', '') as y"],
        field_map=['identity', 'start_date', 'len', 'split(y) as y'], alias="t1",
        f=["type='XTL'", "and", "key='sales'"])
    def select_map(table):
        return {"table": table}

    s1 = select_map(table="mytable")
    print('s1:  {}'.format(s1))

    @Map(mapper=['identity', 'start_date', 'len', "concat(',', d1, d2)"], alias="t2")
    @Map(mapper=['identity', 'start_date', 'len', 'd1', 'd2'], alias="t2")
    def mapper():
        sql = select_map("mytable")
        return sql

    s2 = mapper()
    print('s2:  {}'.format(s2))

In [23]:
test_SelectMap_and_Map()

---------------------
| SelectMap and Map
---------------------
s1:  select identity,start_date,len,split(y) as y from (select identity,start_date,len,regexp_replace(regexp_replace(y, '\\]', ''), '\\[', '') as y from mytable WHERE type='XTL' and key='sales') t1
s2:  select identity,start_date,len,concat(',', d1, d2) from (select identity,start_date,len,d1,d2 from (select identity,start_date,len,split(y) as y from (select identity,start_date,len,regexp_replace(regexp_replace(y, '\\]', ''), '\\[', '') as y from mytable WHERE type='XTL' and key='sales') t1) t2) t2
